In [1]:
from __future__ import annotations

import heapq
import math
import os
import re
from collections import defaultdict, deque
from itertools import combinations

import aocd
import numpy as np
from IPython.display import HTML
from scipy.ndimage import convolve
from tqdm.notebook import tqdm, trange

from shapely.geometry import Polygon, box

import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

from z3 import Int, Optimize, sat

In [2]:
p = aocd.get_puzzle(year=2025, day=10)
p.examples[0].input_data

'[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}\n[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}\n[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}'

In [3]:
def get_data(test_data: bool = False):
    if test_data:
        data = p.examples[0].input_data
    else:
        data = p.input_data
    return data

In [4]:
def process_data(data):
    data = data.split("\n")
    target = []
    tools = []
    jolt = []

    for row in data:
        _target = re.findall(r'\[(.*?)\]', row)
        _tools   = re.findall(r'\((.*?)\)', row)
        _jolt   = re.findall(r'\{(.*?)\}', row)
        _jolt = list(map(int, _jolt[0].split(',')))
        _target = np.array([1 if c == '#' else 0 for c in _target[0]])
        _tools = [tuple(map(int, t.split(','))) for t in _tools]
        
        target.append(_target)
        tools.append(_tools)
        jolt.append(_jolt)
    return target, tools, jolt

# Part 1

In [11]:
def solve_lights_bfs(target, tools):
    """
    Find minimum moves to go from start to target.
    States are tuples of 0s and 1s.
    Returns: list of tool indices used (in order)
    """
    start = tuple(np.array([0] * len(targets[i])))
    target = tuple(target)
    n_lights = len(start)
    
    # Apply a tool to a state
    def apply_tool(state, tool):
        state = list(state)
        for pos in tool:
            state[pos] ^= 1  # Toggle
        return tuple(state)
    
    # BFS
    queue = deque([(start, [])])  # (current_state, moves_taken)
    visited = {start}
    
    while queue:
        state, moves = queue.popleft()
        
        if state == target:
            return moves
        
        for tool_idx, tool in enumerate(tools):
            new_state = apply_tool(state, tool)
            
            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, moves + [tool_idx]))
    
    return None  # No solution

In [12]:
data = get_data(test_data=False)
targets, tools, jolts = process_data(data)

In [13]:
%%time
res = 0
for i in trange(len(targets)):
    start = np.array([0] * len(targets[i]))
    target = targets[i]
    tools_set = tools[i]
    
    solution = solve_lights_bfs(target, tools_set)
    
    if solution is not None:
        res += len(solution)
    else:
        print("No solution exists")

res

  0%|          | 0/199 [00:00<?, ?it/s]

CPU times: user 117 ms, sys: 5.53 ms, total: 123 ms
Wall time: 123 ms


517

# Part 2

In [14]:
data = get_data(test_data=False)
targets, tools, jolts = process_data(data)

In [15]:
def solve_counters_z3(target, tools):
    """
    Solve using Z3 SMT solver.
    """
    n_tools = len(tools)
    n_positions = len(target)
    
    # Create integer variables
    x = [Int(f"x_{i}") for i in range(n_tools)]
    
    # Create optimizer (Z3's optimization extension)
    opt = Optimize()
    
    # Constraint: all variables >= 0
    for i in range(n_tools):
        opt.add(x[i] >= 0)
    
    # Constraint: each position must reach target
    for pos in range(n_positions):
        tools_sum = sum(x[i] for i, tool in enumerate(tools) if pos in tool)
        opt.add(tools_sum == target[pos])
    
    # Objective: minimize total moves
    total_moves = sum(x)
    opt.minimize(total_moves)
    
    # Solve
    if opt.check() == sat:
        model = opt.model()
        tool_counts = [model[x[i]].as_long() for i in range(n_tools)]
        return {
            "tool_counts": tool_counts,
            "moves": sum(tool_counts),
            "valid": True
        }
    return None

In [16]:
%%time

res = 0

for m in trange(len(targets)):
    target = list(jolts[m])
    tool = tools[m]

    solution = solve_counters_z3(target, tool)
    
    if solution:
        res += solution['moves'] 
    else:
        print("No solution exists");

res

  0%|          | 0/199 [00:00<?, ?it/s]

CPU times: user 456 ms, sys: 9.35 ms, total: 465 ms
Wall time: 475 ms


21469